# Montgomery County OH, Risk-Limiting Audit of 2020 Primary

This notebook provides some simple guidance and a way to document the risk-limiting audit in Montgomery County Ohio of the [2020 Primary election, March 17, 2020](https://montgomerycountyoh.epulseadmin2.com/election_results/30/preview?import_id=69439)

See also the [Principles and Best Practices for Post\-Election Tabulation Audits \| ElectionAudits\.org](https://electionaudits.org/principles/)

## Before the audit, publish election results and manifest to be audited

Election results as of 2020-05-11: https://r7j7u2j8.rocketcdn.me/wp-content/uploads/2020/05/03172020es-Official-Final-With-Write-ins.pdf

Ballot manifest: _**TODO**_

Commit to the elections results by publishing this document, tweeting a hash of it, etc.

In [1]:
# Establish 10% risk limit:
risk_limit = 0.1

## Next, roll dice to establish random seed:

Random seed (20 digits): _**TODO**_

## Estimate round sizes

Initialize software, load election results data, and show round sizes for a 70%, 80%, and 90% chance of finishing the audit in the round.

In [2]:
import pandas as pd
import json
from IPython.core.display import display, HTML

# For the Athena calculations, we either import locally (after the repo was cloned)
# or we first clone it and then use it (e.g., when run in Google Colab)
shell = get_ipython().__class__.__name__ 

if shell == 'Shell':
    # imports when launched in e.g., Google Colab
    !git clone https://github.com/filipzz/athena.git r2b2
    from r2b2.code.athena.athena import AthenaAudit
    from r2b2.code.athena.contest import Contest
    from r2b2.code.athena.audit import Audit
else: # shell ==  'ZMQInteractiveShell' or shell == 'TerminalInteractiveShell'
    # local imports if you run it with e.g., Jupyter
    from athena.athena import AthenaAudit
    from athena.contest import Contest
    from athena.audit import Audit

In [3]:
# Results, based on data at https://r7j7u2j8.rocketcdn.me/wp-content/uploads/2020/05/03172020es-Official-Final-With-Write-ins.pdf
# Or URL
results_file = "athena/test_data/2020_montgomery-0511-formatted.json"

In [4]:
results = json.load(open(results_file, 'r'))

In [5]:
results['total_ballots']

69743

In [6]:
# For each contest, display reported results and sample sizes
htmlout = ""
for contest in results['contests']:
    htmlout += f"<H1>Contest: {contest}</H1>\n"
    w = Audit("athena", risk_limit)

    w.read_election_results(results_file)

    w.load_contest(contest)
    htmlout += w.show_election_results().render()

    round_sizes = w.predict_round_sizes([.7, .8, .9])
    df_rs = pd.DataFrame({f'{pstop:.0%}': ss for pstop, ss in round_sizes}, index=['Sample size'])
    htmlout += "<p>Sample sizes by stopping probability:" + df_rs.to_html()

display(HTML(htmlout))

,Candidates,Results
0,Bennet,51
1,Biden,29011
2,Bloomberg,702
3,Buttigieg,525
4,Gabbard,137
5,Klobuchar,406
6,Patrick,27
7,Sanders,5713
8,Steyer,62
9,Warren,1118


## Enter the election data and random seed into Arlo

Publish the ballot selection information from Arlo

## Set up a contest

In [7]:
contest = 'd_president'

In [8]:
w = Audit("athena", risk_limit)
w.read_election_results(results_file)
w.load_contest(contest)

## Tally the first round of samples and check the results
See if the evidence supports finishing the audit.

Do this for each contest.

Enter the sample tally data below for the first round

_**Note, this is just EXAMPLE DATA from test11**_

In [9]:
w.set_observations(200, 100, [0,60,2,2,0,0,0,30,0,6,0])
w.present_state()



	Round: 1 audit failed
	LR:		0.07865222182791687	[needs to be > 1]
	Delta:		12.714198998572465	[needs to be < 1]
	p-value:	0.0010301588480453989	[needs to be <= 0.1]
	both conditions are required to be satisfied.


,Candidates,Results,Round 1,Total,Required
0,Bennet,51,0,0,
1,Biden,29011,60,60,62
2,Bloomberg,702,2,2,
3,Buttigieg,525,2,2,
4,Gabbard,137,0,0,
5,Klobuchar,406,0,0,
6,Patrick,27,0,0,
7,Sanders,5713,30,30,
8,Steyer,62,0,0,
9,Warren,1118,6,6,


**FIXME**: Hmm - the test results are a bit different, e.g. a delta in test11 of 15.63488304 (LR of 0.06395) vs LR = 0.0787 here

## Continue until audit is completed

If there isn't enough evidence yet to complete the audit, pull more ballots and enter more observations as in the last cell.

Enter the incremental data from each round, not cumulative results.

In [10]:
round_sizes = w.predict_round_sizes([.7, .8, .9])

In [11]:
round_sizes

[[0.7, 115], [0.8, 125], [0.9, 138]]

In [12]:
w.set_observations(200, 100, [0,70,1,1,0,0,0,23,0,5,0])
w.present_state()



	Audit Successfully completed!
	LR:		2513.0443937554787	[needs to be > 1]
	p-value:	6.441264615738567e-06	[needs to be <= 0.1]


,Candidates,Results,Round 1,Round 2,Total,Required
0,Bennet,51,0,0,0,
1,Biden,29011,60,70,130,126
2,Bloomberg,702,2,1,3,
3,Buttigieg,525,2,1,3,
4,Gabbard,137,0,0,0,
5,Klobuchar,406,0,0,0,
6,Patrick,27,0,0,0,
7,Sanders,5713,30,23,53,
8,Steyer,62,0,0,0,
9,Warren,1118,6,5,11,


## Publish and share this notebook

Incorporate the final Arlo audit report also

# A different example

## Set up a contest

In [13]:
contest = 'r_senator'

In [14]:
w = Audit("athena", risk_limit)
w.read_election_results(results_file)
w.load_contest(contest)

## Tally the first round of samples and check the results
See if the evidence supports finishing the audit.

Do this for each contest.

Enter the sample tally data below for the first round

_**Note, this is just EXAMPLE DATA....**_

In [15]:
w.set_observations(115, 30, [16, 5, 9])
w.present_state()



	Round: 1 audit failed
	LR:		1.538908805781586	[needs to be > 1]
	Delta:		0.64981108447951	[needs to be < 1]
	p-value:	0.126973980090786	[needs to be <= 0.1]
	both conditions are required to be satisfied.


,Candidates,Results,Round 1,Total,Required
0,Antani,14866,16,16,17
1,Robinson,2885,5,5,
2,Selby,5317,9,9,
3,,Sum,30,,
4,,LR,1.5389,,
5,,P-Value,0.1270,,


## Continue until audit is completed

If there isn't enough evidence yet to complete the audit, pull more ballots and enter more observations as in the last cell.

Enter the incremental data from each round, not cumulative results.

In [16]:
round_sizes = w.predict_round_sizes([.7, .8, .9])

In [17]:
round_sizes

[[0.7, 129], [0.8, 161], [0.9, 202]]

In [18]:
w.set_observations(202, 77, [37, 10, 20])
w.present_state()



	Audit Successfully completed!
	LR:		7.018633183315828	[needs to be > 1]
	p-value:	0.015665256319630765	[needs to be <= 0.1]


,Candidates,Results,Round 1,Round 2,Total,Required
0,Antani,14866,16,37,53,52
1,Robinson,2885,5,10,15,
2,Selby,5317,9,20,29,
3,,Sum,30,77,,
4,,LR,1.5389,7.0186,,
5,,P-Value,0.1270,0.0157,,
